In [1]:
import pandas as pd
import os

In [2]:
INPUT_PATH = r"../artifact/preprocess/"
TRANSLATED_RESULT_PATH = r"../downloads/backup"
ARTIFACT_PATH = r"../artifact/translated_reviews/"

In [3]:
preprocessed_with_id_path = os.path.join(INPUT_PATH, 'preprocessed_reviews.csv')
preprocessed_path = os.path.join(INPUT_PATH, 'all_portuguess.txt')

p2e_path = os.path.join(TRANSLATED_RESULT_PATH, 'trans-p2e.txt')
e2k_path = os.path.join(TRANSLATED_RESULT_PATH, 'trans-e2k.txt')

os.path.exists(preprocessed_path)

True

In [4]:
import csv
translated_result_df = []
for path in [preprocessed_path, p2e_path, e2k_path]:
    df = pd.read_csv(
        path,
        header=None,           # 컬럼명이 없으므로
        names=["review"],      # 컬럼명을 review로 지정
        sep=r'\n',             # 줄바꿈을 한 행 경계로 사용
        engine='python',       # 정규표현식 sep 지원
        quotechar='"',         # 큰따옴표 안의 콤마 무시
        quoting=csv.QUOTE_MINIMAL
    )
    translated_result_df.append(df)

results = pd.concat(translated_result_df, axis=1)
results.columns = ['origin', 'p2e', 'e2k']
# results.head()

In [5]:
origin_with_id_df = pd.read_csv(preprocessed_with_id_path)
# origin_with_id_df

In [6]:
translation_df = origin_with_id_df.copy()
translation_df

# 딕셔너리로 변환
for target in ['p2e', 'e2k']:
    trans_dict = dict(zip(results["origin"], results[target]))

    translation_df[f"title_{target}"]   = translation_df["review_comment_title"].map(trans_dict)
    translation_df[f"message_{target}"] = translation_df["review_comment_message"].map(trans_dict)

os.makedirs(ARTIFACT_PATH, exist_ok=True)
save_path = os.path.join(ARTIFACT_PATH, 'reviews.csv')
translation_df.to_csv(save_path, index=False)